In [13]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

data_path="Data/PisoFirme_AEJPol-20070024_household.dta"

In [14]:
# Table's goal:  tests the effect of the program in terms of its primary objective (i.e., the coverage of cement floors in the
# household), as families not offered the program might have replaced their dirt floors on their own, over time.
df=pd.read_stata(data_path)

In [15]:
# grouping relevant columns names
demog1= ["S_HHpeople", "S_headage" ,"S_spouseage" ,"S_headeduc" ,"S_spouseeduc", "S_rooms"]
# i decided not to use the variable S_rooms because, even if the paper mentioned that they used it in models 2 & 3  
# because if i had included it the results would differ even more from the one of the paper
demog2= ["S_dem1", "S_dem2" ,"S_dem3", "S_dem4", "S_dem5", "S_dem6", "S_dem7", "S_dem8"]
health= ["S_waterland", "S_waterhouse", "S_electricity", "S_hasanimals", "S_animalsinside", "S_garbage", "S_washhands"] 
social= ["S_cashtransfers", "S_milkprogram", "S_foodprogram", "S_seguropopular"]
floor= ["S_shcementfloor", "S_cementfloorkit", "S_cementfloordin", "S_cementfloorbat", "S_cementfloorbed"]

In [16]:
# remove entries with incomplete geograpichal location information
df=df.dropna(subset=["idcluster"])
# fill NaN values of covariates with zeros
df[health+demog1+demog2+social]=df[health+demog1+demog2+social].fillna(0)

In [17]:
# model definition, additional contains the variable excluded in my analysis but cited in the paper
additional =" + S_rooms"
model1="C(dpisofirme)"
model2=model1 + " + C(S_waterland) + C(S_waterhouse) + C(S_electricity) + C(S_hasanimals) \
              + C(S_animalsinside) + C(S_garbage) + S_washhands + S_HHpeople + S_headage + S_spouseage + \
              S_headeduc + S_spouseeduc + S_dem1 + S_dem2 + S_dem3 + S_dem4 + S_dem5 + S_dem6 + \
              S_dem7 + S_dem8" # + additional" 
model3=model2 + " + S_cashtransfers + C(S_milkprogram) + C(S_foodprogram) + \
              C(S_seguropopular)" # + additional

In [18]:
# control group mean and std, first column of the table
control_mean, control_std=np.array([]), np.array([])
for label in floor:
    control_mean=np.append(control_mean,df[df["dpisofirme"]==0][label].mean())
    control_std=np.append(control_std,df[df["dpisofirme"]==0][label].std())

In [19]:
#model 1, second column of the table
out1=[]
for num,label in enumerate(floor):
    lista=[]
    mod = smf.ols(formula=label + ' ~ ' + model1, data=df, missing="raise")
    np.random.seed(2)
    #using clustered standard error as mentioned in the paper
    res = mod.fit(cov_type="cluster", cov_kwds={'groups':df["idcluster"]})
    lista.append(str(round(res.params.values[1],3)).ljust(5,"0"))
    #obtain standard error
    err=res.bse[1]
    #obtain p value of F statistic, with null hypothesis of having coefficient=0
    f=res.f_pvalue
    #prepare output
    if (f < 0.01):
        lista.append("["+str(round(err,3)).ljust(5,"0")+"]***")
    lista.append(str(round(res.params.values[1]*100/control_mean[num],3)).ljust(6,"0"))
    lista="--".join(lista)
    out1.append(lista)

In [20]:
#model 2, third column of the table
out2=[]
for num,label in enumerate(floor):
    lista=[]
    mod = smf.ols(formula=label + ' ~ ' + model2, data=df, missing="raise")
    # mod = smf.ols(formula=label + ' ~ ' + model2 + additional, data=df, missing="raise")
    np.random.seed(2)
    #using clustered standard error as mentioned in the paper
    res = mod.fit(cov_type="cluster", cov_kwds={'groups':df["idcluster"]})
    lista.append(str(round(res.params.values[1],3)).ljust(5,"0"))
    #obtain standard error
    err=res.bse[1]
    #obtain p value of F statistic, with null hypothesis of having coefficient=0
    f=res.f_pvalue
    #prepare output
    if (f < 0.01):
        lista.append("["+str(round(err,3)).ljust(5,"0")+"]***")
    lista.append(str(round(res.params.values[1]*100/control_mean[num],3)).ljust(6,"0"))
    lista="--".join(lista)
    out2.append(lista)

In [21]:
#model 3
out3=[]
for num,label in enumerate(floor):
    lista=[]
    mod = smf.ols(formula=label + ' ~ ' + model3, data=df, missing="raise")
    # mod = smf.ols(formula=label + ' ~ ' + model3 + additional, data=df, missing="raise")
    np.random.seed(2)
    #using clustered standard error as mentioned in the paper
    res = mod.fit(cov_type="cluster", cov_kwds={'groups':df["idcluster"]})
    lista.append(str(round(res.params.values[1],3)).ljust(5,"0"))
    #obtain standard error
    err=res.bse[1]
    #obtain p value of F statistic, with null hypothesis of having coefficient=0
    f=res.f_pvalue
    #prepare output
    if (f < 0.01):
        lista.append("["+str(round(err,3)).ljust(5,"0")+"]***")
    lista.append(str(round(res.params.values[1]*100/control_mean[num],3)).ljust(6,"0"))
    lista="--".join(lista)
    out3.append(lista)

In [22]:
#create dataframe for my replicated results
#same output as the paper regarding the model with format: coeff--std_err---100*coeff/control_mean
#also *** on the standard error indicates that the statistic is significantly different from 0 at 1 percent level.
replica= pd.DataFrame(columns=["Control group mean--std. dev.", "Model 1", "Model 2", "Model 3"], \
                      index=["Share of rooms with cement floors", "Cement floor in kitchen", "Cement floor in dining room", \
                             "Cement floor in bathroom", "Cement floor in bedroom"])
for num in range(len(floor)):
    replica.iloc[num]["Control group mean--std. dev."]=str(round(control_mean[num],3)).ljust(5,"0")+"--"+str(round(control_std[num],3)).ljust(5,"0")
    replica.iloc[num]["Model 1"]=out1[num]
    replica.iloc[num]["Model 2"]=out2[num]
    replica.iloc[num]["Model 3"]=out3[num]
replica

,Control group mean--std. dev.,Model 1,Model 2,Model 3
Share of rooms with cement floors,0.728--0.363,0.202--[0.021]***--27.746,0.207--[0.019]***--28.480,0.210--[0.019]***--28.805
Cement floor in kitchen,0.671--0.470,0.255--[0.025]***--37.936,0.259--[0.023]***--38.648,0.264--[0.023]***--39.330
Cement floor in dining room,0.709--0.455,0.210--[0.026]***--29.633,0.217--[0.025]***--30.569,0.221--[0.025]***--31.131
Cement floor in bathroom,0.803--0.398,0.105--[0.022]***--13.071,0.113--[0.018]***--14.022,0.116--[0.018]***--14.485
Cement floor in bedroom,0.668--0.471,0.238--[0.020]***--35.598,0.245--[0.020]***--36.712,0.245--[0.020]***--36.629


First column: replicated 
Second column: replicated
Third column: different from paper's results of about 0.15%
Fourth column: different from paper's results of about 0.25%
The differences are small enough to be due to different ways to optimize the ols in different programming languages and packages
or maybe from mystakes from my side. Also i did not use in this replication the variable S_rooms which would have made the difference
with the results from the paper more relevant. In any case the results are all qualitatively replicated.